In [1]:
import os
from pathlib import Path
%load_ext autoreload
%autoreload 2


In [3]:
import sys
sys.path.append("../")

In [9]:
import datageneration

In [12]:
len({1,2,3} &{1,2})


2

In [26]:
from datageneration.generate_utils import validate_directory_trees

In [ ]:
def validate_directory_trees(dir1, dir2) -> bool:
    """
    Compare two directories recursively. Files in each directory are
    assumed to be equal if their names and contents are equal.

    Args:
        dir1 (Path):First directory path
        dir2 (Path): Second directory path

    Returns:
        bool: True if the directory trees are the same and
        there were no errors while accessing the directories or files,
        False otherwise.
    """
    dirs_cmp = filecmp.dircmp(dir1, dir2)
    if (
        len(dirs_cmp.common_files) != 0 
        or len(dirs_cmp.left_only) > 0
        or len(dirs_cmp.right_only) > 0
        or len(dirs_cmp.funny_files) > 0
    ):
        return False
    for common_dir in dirs_cmp.common_dirs:
        new_dir1 = dir1 / common_dir
        new_dir2 = dir2 / common_dir
        if not validate_directory_trees(new_dir1, new_dir2):
            return False
    return True


In [28]:
validate_directory_trees("../datageneration/dataset/HR/121374591.hd/","../datageneration/dataset/LR/195913058.hd//")

False

In [19]:
from pathlib import Path
Path("./").exists()

True

In [65]:
import filecmp

def same_folders(dcmp):
    if dcmp.diff_files or dcmp.left_only or dcmp.right_only:
        return False
    for sub_dcmp in dcmp.subdirs.values():
        if not same_folders(sub_dcmp):
            return False
    return True

len(filecmp.dircmp("../datageneration/dataset/HR/121374591.hd/","../datageneration/dataset/LR/191379415.hd/").common_files)
dir1= "../datageneration/dataset/HR/121374591.hd/"
dir2 = "../datageneration/dataset/LR/191379415.hd//"

In [66]:
len(filecmp.dircmp(dir1,dir2).common_files)


63

In [59]:
(_, mismatch, errors) = filecmp.cmpfiles(
    dir1, dir2, filecmp.dircmp(dir1,dir2).common_files, shallow=True
)

In [60]:
len(mismatch)

0

In [21]:
filecmp.dircmp("../datageneration/dataset/HR/121374591.hd/","../datageneration/dataset/HR/121374591.hd/").diff_files

[]

In [ ]:
def compare_directory(dir1,dir2):
    

In [17]:
for i in  os.walk("../../../video-datasets/vimeo/sequences"):
    print(i)
#     break

In [1]:
# for i in os.scandir("../../../video-datasets/vimeo/sequences"):
#     print(i.path)
#     if i.is_file():
#         pass
#     elif i.is_dir():
#         print(os.listdir(i.path))
    

In [7]:
def wrapper(*args, **kwargs):
    args_new =()
    for i in args:
        if type(i)==str:
            args_new.add(Path(i))
        else:
            args_new.add(i)
    for key,value in kwargs.item():
        if type(value)==str:
            kwargs[key]=Path(i)
        else:
            kwargs[key] =value
#     return func(*args, **kwargs).lower()

In [8]:
wrapper("vinod")

<class 'tuple'>


In [38]:
                                                                                              
def list_files(dir):                                                                                                  
    r = []                                                                                                            
    subdirs = [x[0] for x in os.walk(dir)]                                                                            
    for subdir in subdirs:                                                                                            
        files = next(os.walk(subdir))[2]                                                                             
        if (len(files) > 0):                                                                                          
            for file in files:                                                                                        
                r.append(os.path.join(subdir, file))                                                                         
    return r           

In [54]:
# Path("../../../video-datasets/vimeo/sequences").

In [112]:
Path("./datasets/hr").mkdir(parents=True, exist_ok=True)

In [98]:
def run_fast_scandir(dir, ext=[]):
    subfolders, files = [], []
    groups = []

    for f in os.scandir(dir):
        if f.is_dir():
            subfolders.append(f.path)
        if f.is_file():
            if os.path.splitext(f.name)[1].lower() in ext or len(ext) == 0:
                files.append(f.path)

    for dir in list(subfolders):
        _, f, _ = run_fast_scandir(dir, ext)
        groups.append(f)
    return subfolders, files, groups

In [108]:

folder = Path("../../../video-datasets/vimeo/sequences/00026/0001")
subfolders, files,groups = run_fast_scandir(folder, [".jpg",".png",".mp4"])


In [88]:
# subfolders,files,groups

In [110]:
files

['../../../video-datasets/vimeo/sequences/00026/0001/im1.png',
 '../../../video-datasets/vimeo/sequences/00026/0001/im2.png',
 '../../../video-datasets/vimeo/sequences/00026/0001/im3.png',
 '../../../video-datasets/vimeo/sequences/00026/0001/im4.png',
 '../../../video-datasets/vimeo/sequences/00026/0001/im5.png',
 '../../../video-datasets/vimeo/sequences/00026/0001/im6.png',
 '../../../video-datasets/vimeo/sequences/00026/0001/im7.png']

''

In [34]:
l="../../../video-datasets/vimeo/sequences/00001/0001/".split("/")

In [83]:
Path("../../../video-datasets/vimeo/sequences/00001/0001/file.jpg")/{.png}

SyntaxError: invalid syntax (<ipython-input-83-ce880f7f4960>, line 1)

In [30]:
import sys
sys.path.append("../datageneration/")
import generate_utils
import re
re.sub('[^A-Za-z0-9]+', '', "")
from PIL import Image
import numpy as np
import torchvision.transforms.functional as TF
import logging
from tqdm import tqdm

In [9]:
class DataGenerator:
    def __init__(self, **kwargs):
        self.cleanup_factor = kwargs.get("cleanup_factor", 1)
        self.upscale_factor = kwargs.get("upscale_factor", 2)
        self.compress = kwargs.get("compress", False)
        self.ignore_directory = kwargs.get("ignore_directory", False)
        self.ignore_video = kwargs.get("ignore_video", False)
        self.output_directory = Path(kwargs.get("output_directory", "./datasets"))
        self.extensions = kwargs.get("extensions", [])

        self.hr_dir = self.output_directory / "HR"
        self.lr_dir = self.output_directory / "LR"
        self.lr_dir.mkdir(parents=True, exist_ok=True)
        self.hr_dir.mkdir(parents=True, exist_ok=True)

    def process_video(self, video_path):
        if self.ignore_video or not generate_utils.is_video(video_path):
            return
        else:
            pass

    def process_directory(self, directory_path,include_root_name=False):
        # folder supre res
        # every sequence
        # clean image
        # store
        if self.ignore_directory:
            return
        directory_path = generate_utils.change_pathlib(directory_path)
        _, files, groups = generate_utils.run_fast_scandir(
            directory_path, self.extensions
        )
        if len(groups)==0:
            groups.append(files)
        
        for group in tqdm(groups,desc="generating data"):
            self.process_group(group,include_root_name,True)

    def process_group(self, image_array,include_root_name=False,child=False):
        quality = np.random.randint(3, 10) * 10
        group_list = list(filter(lambda x:x, map(lambda x:re.sub(r'[^A-Za-z0-9]', '', x), generate_utils.change_pathlib(image_array[0]).parts)))[:-1]
        if not include_root_name:
            group_list = group_list[1:]
        group_name = '-'.join([str(x) for x in group_list])
        for file in tqdm(image_array,desc=f"generating groups {group_name}",leave=not child):
            if not generate_utils.is_image_file(file) or generate_utils.is_video(file):
                self.process_video(file)
                continue
            file = generate_utils.change_pathlib(file)
            try:   

                group_name = Path(group_name)
                input_img = Image.open(file).convert("RGB")
                input_img = TF.to_tensor(input_img)
                input_img = generate_utils.imresize(
                    input_img, 1.0 / self.cleanup_factor, True
                )
                hr_path = self.hr_dir / group_name
                lr_path = self.lr_dir / group_name
                hr_path.mkdir(parents=True,exist_ok=True)
                lr_path.mkdir(parents=True,exist_ok=True)
                TF.to_pil_image(input_img).save(
                    str(f"{hr_path/file.stem}.png"), "PNG"
                )
                resize_img = generate_utils.imresize(
                    input_img, 1.0 / self.upscale_factor, True
                )

                if self.compress:
                    TF.to_pil_image(resize_img).save(
                        str(f"{lr_path/file.stem}.jpg"),
                        "JPEG",
                        quality=quality,
                    )
                else:
                    TF.to_pil_image(resize_img).save(
                        str(f"{lr_path/file.stem}.png"), "PNG"
                    )
            except Exception as e:
                logging.error(f"error while processing {file} the error is {e}")

    def process(self, dataset_path,include_root_name=False):
        for i in os.scandir(dataset_path):
            if i.is_file():
                self.process_video(i.path)
            elif i.is_dir():
                self.process_directory(i.path,include_root_name)

In [18]:
a=torch.tensor([1])
f"{a.size()[0]}"

'1'

In [58]:
d=DataGenerator(compress=True)
d.process("../../../video-datasets/vimeo_super_resolution_test/low_resolution/00001/")

generating data:   0%|          | 0/75 [00:00<?, ?it/s]

  0%|          | 0/7 [00:00<?, ?it/s]

                                     

  0%|          | 0/7 [00:00<?, ?it/s]

generating data:   3%|▎         | 2/75 [00:00<00:07, 10.23it/s]

  0%|          | 0/7 [00:00<?, ?it/s]

                                     

  0%|          | 0/7 [00:00<?, ?it/s]

generating data:   5%|▌         | 4/75 [00:00<00:06, 10.32it/s]

  0%|          | 0/7 [00:00<?, ?it/s]

                                     

  0%|          | 0/7 [00:00<?, ?it/s]

100%|██████████| 7/7 [00:00<00:00, 64.53it/s]

generating data:   8%|▊         | 6/75 [00:00<00:06,  9.96it/s]

  0%|          | 0/7 [00:00<?, ?it/s]

                                     

  0%|          | 0/7 [00:00<?, ?it/s]

100%|██████████| 7/7 [00:00<00:00, 68.84it/s]

generating data:  11%|█         | 8/75 [00:00<00:06,  9.92it/s]

  0%|          | 0/7 [00:00<?, ?it/s]

                                     

  0%|          | 0/7 [00:00<?, ?it/s]

gene

KeyboardInterrupt: 